## Hyperparameter tuning of the server-side optimizer with Optuna

This notebook shows specifically how to tune the *learning rate* of *FedAdam* using the Optuna package. Tuning of other hyperparameter and/or other server-side optimizers can be done analogously. The notebook *Aggregators.ipynb* shows how to use different aggregators with the FEDn Python API.

For a complete list of implemented interfaces, please refer to the [FEDn APIs](https://fedn.readthedocs.io/en/latest/fedn.network.api.html#module-fedn.network.api.client). 

For implementation details related to how aggregators are implemented, we recommend to read [FEDn Framework Extensions](https://www.scaleoutsystems.com/post/fedn-framework-extensions).

Before starting this tutorial, make sure you have a project running in FEDn Studio and have created the compute package and the initial model. If you're not sure how to do this, please follow the instructions in sections 1, 2, and 3 of the [quickstart guide](https://fedn.readthedocs.io/en/latest/quickstart.html). 

In [57]:
from fedn import APIClient
import time
import json
import numpy as np

In this example, we assume the project is hosted on the public FEDn Studio. You can find the CONTROLLER_HOST address in the project dashboard.

**Note:** If you're using a local sandbox, the CONTROLLER_HOST will be "localhost," and the CONTROLLER_PORT will be 8092.

Next, you'll need to generate an access token. To do this, go to the project page in FEDn Studio, click on "Settings," then "Generate token." Copy the access token from the Studio and paste it into the notebook. In case you need further details, have a look at the [FEDn ClientAPIs](https://fedn.readthedocs.io/en/latest/apiclient.html#).

In [58]:
CONTROLLER_HOST = 'fedn.scaleoutsystems.com/<your-project-name>' # TODO byt ut till lokal
ACCESS_TOKEN = '<your-access-token>'
client = APIClient(CONTROLLER_HOST,token=ACCESS_TOKEN, secure=True,verify=True)

Initialize FEDn with the compute package and seed model. Note that these files needs to be created separately. If you're not sure how to do this, please follow the instructions only in section 3 of the [quickstart guide](https://fedn.readthedocs.io/en/latest/quickstart.html#create-the-compute-package-and-seed-model).

In [59]:
client.set_active_package('../mnist-pytorch/package.tgz', 'numpyhelper')
client.set_active_model('../mnist-pytorch/seed.npz')
seed_model = client.get_active_model()

### Defining the objective function

Optuna expects an objective function - the function that evaluates a certain set of hyperparameter values. In this example, we will use the test accuracy as a validation score and we want to maximize it.

For each set of hyperparameter values, each `trial`, we will start a new session using the FEDn Python API. In each session/trial, we will select the model with the highest test accuracy and use that in the Optuna objective function to evaluate the trial.

The `objective()` function gives us some flexibility in how we choose to evaluate the choice of hyperparameters in each trial/session. Below are two examples on how to calculate the attained validation accuracy in a session.

* ``

In [78]:
# Helper function to get the highest test accuracy within a session
def get_highest_test_accuracy_in_session(client, n_rounds):
    best_accuracy = 0
    validations_in_session = client.get_validations()['result'][:n_rounds]
    for validation in validations_in_session:
        val_accuracy = json.loads(validation['data'])['test_accuracy']
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy

    return best_accuracy

# Helper function to get the average test accuracy over the last 10 rounds in a session
def get_test_accuracy_in_session_smooth(client, n_rounds):
    
    n_rounds_to_avg = 5
    if n_rounds_to_avg > n_rounds:
        n_rounds_to_avg = n_rounds
    
    # New
    models = client.get_model_trail()[-n_rounds_to_avg:] # model with index -1 lacks validations -> seed model??
    # print(f'models: {len(models)}\n {models}')
    model_test_acc = []

    # Loop over the last 'n_rounds_to_avg' rounds
    for model_index, model in enumerate(models):
        
        model_id = model["model"]
        validations = client.get_validations(model_id=model_id)
        # print(f'Validation nr. {model_index}: {validations}')
        a = []

        # Loop over all contributing clients
        for validation in validations['result']: 
            metrics = json.loads(validation['data'])
            a.append(metrics['test_accuracy'])
            
        model_test_acc.append(a)
        print(f'Model id: {model_id}, Validations: {validations}')

    mean_val_accuracies = [np.mean(x) for x in model_test_acc]
    print(f'Mean accuracy: {mean_val_accuracies}')

    # Old
    # validations_to_avg = client.get_validations()['result'][:n_rounds_to_avg]
    # val_accuracies = [json.loads(validation['data'])['test_accuracy'] for validation in validations_to_avg]

    mean_val_accuracy = np.mean(mean_val_accuracies)
    print(f'Validation accuracy scores:\n{mean_val_accuracies}')
    print(f'Average validation accuracy: {mean_val_accuracy}')

    return mean_val_accuracy

In [79]:
import optuna

# Objective function which will be sent to Optuna to evaluate the selection of hyperparameter values
def objective(trial):
    # Number of rounds per session
    n_rounds = 5

    # Suggest hyperparameter priors
    learning_rate = trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True)

    # Set session configurations (from seed model)
    session_config = {
                        "helper": "numpyhelper",
                        "aggregator": "fedopt",
                        "aggregator_kwargs": {
                            "serveropt": "adam",
                            "learning_rate": learning_rate
                            },
                        "model_id": seed_model['model'],
                        "rounds": n_rounds
                    }

    # Run session and get session id
    result_fedadam = client.start_session(**session_config)
    session_id = result_fedadam['config']['session_id']
    
    # Wait for the session to finish
    while not client.session_is_finished(session_id):
        time.sleep(2)
    
    # Return validation accuracy for session
    return get_test_accuracy_in_session_smooth(client=client, n_rounds=n_rounds)

### Creating and running an Optuna study

Here we create an Optuna study. Since we are using the test accuracy for evaluation, we want to maximize the objective function in this example. We pass the objective function defined earlier when calling `study.optimize()` and select the number of trials we want to perform.

**Note:** Each trial starts a session, so the number of sessions is `n_trials`.

In [80]:
# Create an Optuna study
study = optuna.create_study(direction="maximize")

# Optimize hyperparameters
study.optimize(objective, n_trials=1)
print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

[I 2024-09-09 11:40:23,930] A new study created in memory with name: no-name-9f40df3c-4c9f-4283-a7df-2608de273a5f


Model id: f8df5900-233b-4ee8-aa27-cbb197f517b4, Validations: {'count': 2, 'result': [{'correlation_id': '765ae8e1-bcfc-41b9-90e5-c47d122139ee', 'data': '{"training_loss": 2.9918696880340576, "training_accuracy": 0.2761666774749756, "test_loss": 3.145538568496704, "test_accuracy": 0.25600001215934753}', 'id': '66dec299819bae1528aeeefd', 'meta': '', 'model_id': 'f8df5900-233b-4ee8-aa27-cbb197f517b4', 'receiver': {'clientId': '', 'name': 'hyperparametertuning-hrt-fedn', 'role': 'COMBINER'}, 'sender': {'clientId': '', 'name': 'client735', 'role': 'WORKER'}, 'session_id': 'cc9be434-4dda-4dc5-b40e-762ff38cefac', 'timestamp': '2024-09-09T09:40:41.057123Z'}, {'correlation_id': '4bbb7175-52d1-4ad5-82cc-b97f8b728d19', 'data': '{"training_loss": 3.0555734634399414, "training_accuracy": 0.26350000500679016, "test_loss": 3.146380662918091, "test_accuracy": 0.25200000405311584}', 'id': '66dec299819bae1528aeeefb', 'meta': '', 'model_id': 'f8df5900-233b-4ee8-aa27-cbb197f517b4', 'receiver': {'clientId'

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
[W 2024-09-09 11:41:29,099] Trial 0 failed with parameters: {'learning_rate': 0.09377062950652325} because of the following error: The value nan is not acceptable.
[W 2024-09-09 11:41:29,100] Trial 0 failed with value nan.


Model id: e4a747bb-f5a2-4845-898d-ca6e9d005bb0, Validations: {'count': 0, 'result': []}
Mean accuracy: [0.2540000081062317, 0.28999999165534973, 0.44349999725818634, 0.6869999766349792, nan]
Validation accuracy scores:
[0.2540000081062317, 0.28999999165534973, 0.44349999725818634, 0.6869999766349792, nan]
Average validation accuracy: nan


ValueError: No trials are completed yet.

### Visualize Optuna's optimization



In [ ]:
import optuna.visualization as vis

vis.plot_slice(study)

In [53]:
vis.plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                   14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
                                   26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
                                   38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                                   50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61,
                                   62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
                                   74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
                                   86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                                   98, 99],
                         'colorbar': {'title': {'text': 'Trial'}, 'x': 1.0, 'xpad': 40},
                         'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                                        'rgb(222,235,247)'], [0.25,
                                        'rgb(198,219,239)'], [0.375,
                                        'rgb(158,202,225)'], [0.5,
                                        'rgb(107,174,214)'], [0.625,
                                        'rgb(66,146,198)'], [0.75,
                                        'rgb(33,113,181)'], [0.875,
                                        'rgb(8,81,156)'], [1.0, 'rgb(8,48,107)']],
                         'line': {'color': 'Grey', 'width': 0.5}},
              'mode': 'markers',
              'name': 'Feasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [0.01060256463073004, 0.06363319752806389, 0.044054634905094,
                    0.005457455596524018, 0.0006534358443990889,
                    0.005409564385563341, 0.000296117238409861,
                    0.0001548659658268065, 0.0059773222630647126,
                    0.009610312272318417, 0.07936629680356555, 0.0999017085272383,
                    0.03030693483792421, 0.09839071999176227,
                    0.0018120798841072265, 0.02630581835365611,
                    0.0015862265658428628, 0.018963170798261392,
                    0.05591498203997656, 0.0507252826898167, 0.015598714196031616,
                    0.04981982237560933, 0.04472041885602553, 0.04569663540155219,
                    0.02409255925048435, 0.012153379377141215, 0.06204357797505839,
                    0.0318047942959313, 0.0026889985534921607,
                    0.008406962615909752, 0.07570103253376405,
                    0.059086197752773524, 0.03714901133834086, 0.0646915046763561,
                    0.018225411751837776, 0.060873750215417036,
                    0.0005937941753721039, 0.03794230073789979,
                    0.021776847720301837, 0.012825763536089739,
                    0.006945279111941595, 0.003924318415667345,
                    0.0009962905683287265, 0.007485623216043211,
                    0.08121672594329421, 0.03218747043584, 0.0043209290517303035,
                    0.09007673305976491, 0.00011242006456928975,
                    0.052926437331853586, 0.015080515584708165,
                    0.06571961786071892, 0.06853902191868609, 0.043188154256233874,
                    0.03137506646247659, 0.09815877477219788, 0.024965815660978004,
                    0.0002369246671375477, 0.05402719364497381,
                    0.04024469007474665, 0.0021520115377091278,
                    0.06777601481845777, 0.054552697374482345, 0.07824127973056091,
                    0.062190956331727144, 0.04574533489252647,
                    0.026700039728034024, 0.035572253575668344,
                    0.020947893979189017, 0.09807853814362605, 0.074008662788353,
                    0.006494329870318023, 0.013324381014246824,
                    0.047348523959182987, 0.05213136025170205, 0.0468237789021135,
                    0.009671712975184507, 0.028449834843744366,
                    0.03736743378716483, 0.0011102163493235233,
                 